# Sentiment Analysis using Naive Bayes

In this assignment, we will attempt to label tweets with sentiments (positive, neutral and negative) using Naive Bayes classifier. Naive Bayes is a very basic approach to this problem, but gives surprisingly good accuracy sometimes.

**Fill in the Blanks**

## Importing required libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import re
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import make_scorer, accuracy_score, f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix, roc_auc_score, recall_score, precision_score,classification_report

In [3]:
%cd drive/'My Drive'/END/

/content/drive/My Drive/END


## Reading dataset

In [4]:
data=pd.read_csv('tweets.csv')
data.drop(data.columns[0],axis=1,inplace=True)
data.head()

,tweets,labels
0,Obama has called the GOP budget social Darwini...,1
1,"In his teen years, Obama has been known to use...",0
2,IPA Congratulates President Barack Obama for L...,0
3,RT @Professor_Why: #WhatsRomneyHiding - his co...,0
4,RT @wardollarshome: Obama has approved more ta...,1


## Text processing for the tweets

In [5]:
import nltk 
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:

from nltk.tokenize import word_tokenize
from string import punctuation 
from nltk.corpus import stopwords 

stopwords = set(stopwords.words('english') + list(punctuation) + ['AT_USER','URL'])
    
def processTweet(tweet):
    # tweet is the text we will pass for preprocessing 
    # convert passed tweet to lower case 
    tweet=str(tweet).lower()
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet) # remove URLs
    tweet = re.sub('@[^\s]+', 'AT_USER', tweet) # remove usernames
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet) # remove the # in #hashtag
    
    # use work_tokenize imported above to tokenize the tweet
    tweet=word_tokenize(tweet)
    return [word for word in tweet if word not in stopwords]

## Process all tweets

In [7]:
processed=[]

for tweet in data['tweets']:
    
    # process all tweets using processTweet function above - store in variable 'cleaned' 
    cleaned=processTweet(tweet)
    processed.append(' '.join(cleaned))

In [8]:
data['processed'] = processed

In [9]:

data.head()


,tweets,labels,processed
0,Obama has called the GOP budget social Darwini...,1,obama called gop budget social darwinism nice ...
1,"In his teen years, Obama has been known to use...",0,teen years obama known use marijuana cocaine
2,IPA Congratulates President Barack Obama for L...,0,ipa congratulates president barack obama leade...
3,RT @Professor_Why: #WhatsRomneyHiding - his co...,0,rt whatsromneyhiding connection supporters cri...
4,RT @wardollarshome: Obama has approved more ta...,1,rt obama approved targeted assassinations mode...


## Create pipeline and define parameters for GridSearch

In [10]:
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB())])

tuned_parameters = {
    'vect__ngram_range': [(1, 1), (1, 2), (2, 2)],
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    'clf__alpha': [1, 1e-1, 1e-2]
}

## Split data into test and train

In [11]:
# split data into train and test with split as 0.2 
X = data.processed
y = data.labels

x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=1)

In [12]:

X[0]

'obama called gop budget social darwinism nice try believe social creationism'

## Perform classification (using GridSearch)

In [13]:
# perform GridSearch CV with 10 fold CV using pipeline and tuned_paramters defined above 
clf = GridSearchCV(text_clf,tuned_parameters,cv=10)
clf=clf.fit(x_train, y_train)

## Classification report 

In [14]:

# print classification report after predicting on test set with best model obtained in GridSearch
prediction=clf.best_estimator_.predict(x_test)
class_report=classification_report(y_test,prediction)
print(class_report)

              precision    recall  f1-score   support

           0       0.81      0.98      0.89       187
           1       0.89      0.59      0.71        71
           2       1.00      0.17      0.29        18

    accuracy                           0.83       276
   macro avg       0.90      0.58      0.63       276
weighted avg       0.85      0.83      0.81       276



## Important:

In [15]:

counts = data.labels.value_counts()
print(counts)

0    947
1    352
2     81
Name: labels, dtype: int64


We can see above that the class distribution is highly imbalanced, this would not lead to good sampling of the data for the classifier. For your learning, try using [SMOTE](https://imbalanced-learn.readthedocs.io/en/stable/api.html) to oversample the minority classes and then evaluate the performance with Naive Bayes and compare.